In [3]:
import pandas as pd
import numpy as np

# Параметры
n_users = 10000
min_interactions = 5
max_interactions = 50
np.random.seed(42)

# Загрузка данных
tracks_df = pd.read_csv("../edadata.csv")

# Нормализуем популярность в вероятности
pop_weights = tracks_df['popularity'].values.astype(float)
# Защита от деления на ноль
if pop_weights.sum() == 0:
    pop_weights = np.ones_like(pop_weights)
pop_weights = pop_weights / pop_weights.sum()

track_ids = tracks_df['item_id'].values

interactions = []

for user_id in range(n_users):
    n_interactions = np.random.randint(min_interactions, max_interactions + 1)
    # Ограничение: не больше, чем доступно треков
    n_sample = min(n_interactions, len(track_ids))
    sampled_items = np.random.choice(
        track_ids,
        size=n_sample,
        replace=False,
        p=pop_weights
    )
    for item_id in sampled_items:
        interactions.append({
            'user_id': user_id,
            'item_id': item_id,
            'relevance': 1,
        })

interactions_df = pd.DataFrame(interactions)

# === ИСПРАВЛЕНИЕ: генерация timestamp без переполнения ===
# Создаём диапазон дат
start = pd.Timestamp('2024-10-26')
end = pd.Timestamp('2025-10-26')

# Генерируем случайные timedelta в пределах года
time_diff = (end - start).total_seconds()
random_seconds = np.random.rand(len(interactions_df)) * time_diff
timestamps = start + pd.to_timedelta(random_seconds, unit='s')

interactions_df['timestamp'] = timestamps

# Сохраняем
interactions_df.to_csv("../interaction_data.csv", index=False)